# Segment Vs Kinesis Analysis

In [1]:
import pgdbConnection

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn=pgdbConnection.get_connection()
cursor = conn.cursor()
cursor.execute("SELECT datname from pg_database")
rows = cursor.fetchall()

Connecting to database
  ->host=data-pier-production.cl8qfdl47mtr.ap-southeast-1.rds.amazonaws.com port=5432 dbname=data_pier user=pari password=ja5DAmTK8hD3zuPpnNaySJJcgYxnLGJD
Connected!



In [3]:
print ("Show me the databases:")
for row in rows:
    print (row[0])

Show me the databases:
template0
rdsadmin
template1
postgres
data_pier


In [4]:
cursor.execute("SELECT COUNT(*),CAST(received_at AS DATE) AS rdt FROM moneysmartsg_prod.lead_generation_click_conversion WHERE CAST(received_at AS DATE) >='2019-12-03' GROUP BY rdt;")
rows = cursor.fetchall()

import pprint
pprint.pprint(rows)

[(167, datetime.date(2019, 12, 6)),
 (245, datetime.date(2019, 12, 9)),
 (243, datetime.date(2019, 12, 4)),
 (186, datetime.date(2019, 12, 5)),
 (182, datetime.date(2019, 12, 10)),
 (120, datetime.date(2019, 12, 11)),
 (192, datetime.date(2019, 12, 8)),
 (153, datetime.date(2019, 12, 7)),
 (47, datetime.date(2019, 12, 12)),
 (239, datetime.date(2019, 12, 3))]


In [1]:
import sys
try:
    from pyathena import connect
except:
    print("Failed to import pyathena, trying to install it")
    !{sys.executable} -m pip install PyAthena
    from pyathena import connect
    print("successfully installed")
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
    
import pandas as pd
import athenaConnection


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Events from Athena

In [2]:
print("Connecting to Athena")
athena = athenaConnection.AthenaQuery()
athena_conn, athena_cursor =athena.connect_to_athena()
athena.connect()


Connecting to Athena


In [11]:
# config values..
event_log_s3_path = "s3://ms-data-pipeline-production/ms-data-stream-production-processed"
event_log_s3_bucket = event_log_s3_path.split("s3://")[1].split("/")[0]
event_log_s3_prefix = event_log_s3_path.split("/")[-1]
athena_bucket_path = "s3://aws-athena-query-results-358002497134-ap-southeast-1/"
#athena_database = "ms_data_processed_production"
athena_database = "data_science"
athena_raw_events_table = "ms_data_stream_production_processed"
#athena_raw_events_table = "ms_data_stream_production_processed_cd5f4696237059a21d780afa83822e6b"
athena_year_partition = "partition_0" #2019
athena_month_partition = "partition_1" #02
athena_day_partition = "partition_2" #0
athena_easy_events_table =  "id_ab_test"
aws_region_name = "ap-southeast-1"


In [22]:
print("Running small test query to check the athena connection")
query = "select * from data_science.ms_data_stream_production_processed limit 10"
#pd.read_sql(query, athena_conn)#using pandas
athena.query(query)#using custom implementation

Running small test query to check the athena connection


,type,context,user,body,sent_at,partition_0,partition_1,partition_2
0,event,"{title=smartwidget_client, page_url=https://ww...",{anonymous_id=5ef3d608-a823-4884-8a91-ccd112d9...,"{event_name=UserView.WidgetLoad, data={title=n...",2019-12-08T11:11:44.909Z,2019,12,8
1,event,{title=KLOOK Promo Code Singapore Guide — Cred...,{anonymous_id=0e85b318-2c9b-4629-9e7c-9012824e...,"{event_name=Reading, data={title=KLOOK Promo C...",2019-12-08T11:11:44.716Z,2019,12,8
2,event,{title=15 Usaha Makanan Unik dan Kreatif yang ...,{anonymous_id=8dbbedea-e917-4ef4-8d74-4acc41b0...,"{event_name=Reading, data={title=15 Usaha Maka...",2019-12-08T11:11:44.013Z,2019,12,8
3,event,{title=MMA Studios in Singapore: Best Places T...,{anonymous_id=767eaaf6-ca3b-4e84-9bdf-5ea74b09...,"{event_name=Reading, data={title=MMA Studios i...",2019-12-08T11:11:44.266Z,2019,12,8
4,event,{title=Fitri Salhuteru dan 6 Kaosnya yang Seha...,{anonymous_id=02e53ae4-52b3-4d95-a81c-4198c662...,"{event_name=Reading, data={title=Fitri Salhute...",2019-12-08T11:11:48.781Z,2019,12,8
5,event,{title=Rincian Lengkap Biaya Tambah Daya Listr...,{anonymous_id=21790dbf-b77a-4934-aa82-399cf081...,"{event_name=Reading, data={title=Rincian Lengk...",2019-12-08T11:11:54.427Z,2019,12,8
6,event,{title=Berapa Gaji Youtuber? Begini Cara Mengh...,{anonymous_id=729c038a-e47c-44b5-b1e5-de66c1be...,"{event_name=Reading, data={title=Berapa Gaji Y...",2019-12-08T11:11:42.534Z,2019,12,8
7,event,{title=【大阪環球影城】USJ入場門票及快證 2019最新價格比較 | MoneySm...,{anonymous_id=13927015-9557-426c-9f52-a9293280...,"{event_name=Reading, data={title=【大阪環球影城】USJ入場...",2019-12-08T11:11:41.811Z,2019,12,8
8,page,"{title=smartwidget_client, page_url=https://ww...",{anonymous_id=5ef3d608-a823-4884-8a91-ccd112d9...,"{event_name=PageView, data={title=null, status...",2019-12-08T11:11:44.852Z,2019,12,8
9,event,{title=Maryono Mantan Dirut BTN Kekayaannya Me...,{anonymous_id=f8ab76be-34a8-4aa3-946f-8e6bbfde...,"{event_name=Reading, data={title=Maryono Manta...",2019-12-08T11:13:17.702Z,2019,12,8
